CNN

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

from google.colab import drive
drive.mount('/content/drive')

# ตั้งที่อยู่ของไฟล์
train_path = '/content/drive/My Drive/Data/Trainning_Set.csv'
test_path = '/content/drive/My Drive/Data/Test_Set.csv'

# อ่านไฟล์ CSV
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# แปลง PM2.5 เป็น Sliding Window
def create_sequences(data, n_steps):
    sequences = []
    labels = []
    for i in range(len(data) - n_steps):
        seq = data[i:i + n_steps]
        label = data[i + n_steps]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# กำหนดจำนวนวันที่ต้องการดูย้อนหลัง
n_steps = 7

# ตรวจสอบให้แน่ใจว่า 'PM2.5' เป็นตัวเลข
train_data['PM2.5'] = pd.to_numeric(train_data['PM2.5'], errors='coerce')
test_data['PM2.5'] = pd.to_numeric(test_data['PM2.5'], errors='coerce')

# ลบแถวที่มีค่า NaN หลังจากแปลง
train_data = train_data.dropna(subset=['PM2.5'])
test_data = test_data.dropna(subset=['PM2.5'])

# เตรียม Training และ Test Set
X_train, y_train = create_sequences(train_data['PM2.5'].values, n_steps)
X_test, y_test = create_sequences(test_data['PM2.5'].values, n_steps)

# ตรวจสอบการจัดการกับ NaN ในชุดข้อมูล
X_train = np.nan_to_num(X_train)  # แปลง NaN เป็น 0
y_train = np.nan_to_num(y_train)
X_test = np.nan_to_num(X_test)
y_test = np.nan_to_num(y_test)

# Reshape สำหรับ CNN ให้เป็น 3D (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # เพิ่มมิติที่ 3 สำหรับ features
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))  # เพิ่มมิติที่ 3 สำหรับ features

# สร้างโมเดล CNN
model_cnn = Sequential([
    Conv1D(64, 2, activation='relu', input_shape=(n_steps, 1)),  # Conv1D ต้องการ input_shape เป็น 3D (samples, time steps, features)
    MaxPooling1D(),
    Conv1D(32, 2, activation='relu'),
    MaxPooling1D(),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer สำหรับค่าตัวเลขเดียว
])

# คอมไพล์โมเดล
model_cnn.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

# เทรนโมเดล
model_cnn.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# ทำนายค่าจาก Test Set
y_pred = model_cnn.predict(X_test)

# คำนวณ MSE, RMSE, MAE, MAPE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)*100

# แสดงผลลัพธ์
print(f'MSE: {mse}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}')

# เพิ่มข้อมูลที่ทำนายล่าสุดในแต่ละวันจาก Test Set เข้า Training Set
test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])  # แปลงเป็น datetime
latest_per_day = test_data.loc[test_data.groupby(test_data['วันที่เต็ม'].dt.date)['วันที่เต็ม'].idxmax()]  # เลือกข้อมูลล่าสุดของแต่ละวัน

# เลือกคอลัมน์ทั้งหมดสำหรับการอัปเดต Training Set
columns_to_include = ['สถานี', 'วันที่เต็ม', 'ปริมาณฝน', 'ความชื้นสัมพัทธ์', 'ความเร็วลม', 'ทิศทางลม', 'หมอก', 'อุณหภูมิ', 'PM10', 'PM2.5']
latest_per_day = latest_per_day[columns_to_include]  # เก็บเฉพาะคอลัมน์ที่ต้องการ

# รวมข้อมูลเข้ากับ Training Set
updated_train_data = pd.concat([train_data[columns_to_include], latest_per_day], ignore_index=True)

# จัดการค่า NaN ใน Training Set ใหม่
updated_train_data = updated_train_data.dropna(subset=['PM2.5'])  # ลบแถวที่ไม่มี PM2.5
updated_train_data.fillna(0, inplace=True)  # แทนที่ค่า NaN อื่น ๆ ด้วย 0

# เตรียมข้อมูลใหม่สำหรับการเทรน
X_train_new, y_train_new = create_sequences(updated_train_data['PM2.5'].values, n_steps)

# แปลงข้อมูลใหม่สำหรับการเทรน
X_train_new = np.nan_to_num(X_train_new)
y_train_new = np.nan_to_num(y_train_new)

# Reshape สำหรับ CNN
X_train_new = X_train_new.reshape((X_train_new.shape[0], X_train_new.shape[1], 1))

# เทรนโมเดลใหม่ด้วย Training Set ที่อัปเดตแล้ว
model_cnn.fit(X_train_new, y_train_new, epochs=10, batch_size=32)

# เพิ่มฟิลด์ Actual และ Predicted สำหรับการประเมิน
results = pd.DataFrame({
    'Actual': y_test.flatten(),
    'Predicted': y_pred.flatten()
})

# เลือกคอลัมน์ที่ต้องการจาก test_data
columns_to_include = ['สถานี', 'วันที่เต็ม', 'ปริมาณฝน', 'ความชื้นสัมพัทธ์', 'ความเร็วลม', 'ทิศทางลม', 'หมอก', 'อุณหภูมิ', 'PM10']

# หาค่าล่าสุดของแต่ละวันในชุดข้อมูล test_data
test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])
latest_per_day = test_data.loc[test_data.groupby(test_data['วันที่เต็ม'].dt.date)['วันที่เต็ม'].idxmax()]

# เลือกเฉพาะคอลัมน์ที่ต้องการ
latest_per_day = latest_per_day[columns_to_include]

# รวมข้อมูลล่าสุดของแต่ละวันเข้ากับผลลัพธ์การทำนาย
final_results = pd.concat([results, latest_per_day.reset_index(drop=True)], axis=1)

# บันทึกผลลัพธ์ที่รวมข้อมูลลงในไฟล์ CSV
final_results.to_csv('/content/drive/My Drive/Data/Predictions_with_Fields_CNN.csv', index=False)

# บันทึก Training Set ที่อัปเดตแล้ว
updated_train_data.to_csv('/content/drive/My Drive/Data/Updated_Training_Set_CNN.csv', index=False)

# บันทึกโมเดลที่ฝึกแล้วเป็นไฟล์ H5
model_cnn.save('/content/drive/My Drive/Data/CNN_Model.h5')

print("ผลลัพธ์การทำนายพร้อมฟิลด์ที่เพิ่มเข้าไปถูกบันทึกแล้ว!")
print("โมเดลได้ถูกบันทึกในไฟล์ H5 แล้ว!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 238.5731 - mae: 11.1891 - mape: 50.0764 - val_loss: 89.3090 - val_mae: 6.5159 - val_mape: 26.8148
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 69.7071 - mae: 5.8449 - mape: 29.1822 - val_loss: 77.6487 - val_mae: 5.9100 - val_mape: 26.3375
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 67.5263 - mae: 5.6428 - mape: 27.5481 - val_loss: 60.7686 - val_mae: 5.3786 - val_mape: 23.1697
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 58.1751 - mae: 5.1654 - mape: 25.6547 - val_loss: 59.9970 - val_mae: 5.3465 - val_mape: 23.7997
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 56.6778 - mae: 5.2886 - mape: 26.7961 - val_loss: 56.2901 - val_mae: 5.1367 - val_mape: 21.7572
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 52.0813 - mae: 5.0139 - mape: 25.4854 - val_loss: 55.7901 - val_mae: 5.1198 - val_mape: 22.2869
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 53.0858 - mae: 5.0980 - ma

<ipython-input-19-4c1cff3be4b7>:84: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])  # แปลงเป็น datetime


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 45.3966 - mae: 4.7314 - mape: 23.6156
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 44.7071 - mae: 4.6608 - mape: 23.3505
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 49.0942 - mae: 4.7749 - mape: 23.4690
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 45.3184 - mae: 4.7571 - mape: 24.2767
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 45.2772 - mae: 4.7714 - mape: 23.5249
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51.4589 - mae: 4.9966 - mape: 24.5844
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 54.7743 - mae: 4.8648 - mape: 22.8433
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 44.8787 - mae: 4.6394 - mape: 22.8541
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 45.7708 - mae: 4.6564 - mape: 22.8865
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 44.4253 - mae: 4.6851 - mape: 23.1727


ผลลัพธ์การทำนายพร้อมฟิลด์ที่เพิ่มเข้าไปถูกบันทึกแล้ว!
โมเดลได้ถูกบันทึกในไฟล์ H5 แล้ว!


RNN

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from google.colab import drive

# เชื่อมต่อกับ Google Drive
drive.mount('/content/drive')

# ตั้งที่อยู่ของไฟล์
train_path = '/content/drive/My Drive/Data/Trainning_Set.csv'
test_path = '/content/drive/My Drive/Data/Test_Set.csv'

# อ่านไฟล์ CSV
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# แปลง PM2.5 เป็น Sliding Window
def create_sequences(data, n_steps):
    sequences = []
    labels = []
    for i in range(len(data) - n_steps):
        seq = data[i:i + n_steps]
        label = data[i + n_steps]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# กำหนดจำนวนวันที่ต้องการดูย้อนหลัง
n_steps = 7

# ตรวจสอบให้แน่ใจว่า 'PM2.5' เป็นตัวเลข
train_data['PM2.5'] = pd.to_numeric(train_data['PM2.5'], errors='coerce')
test_data['PM2.5'] = pd.to_numeric(test_data['PM2.5'], errors='coerce')

# ลบแถวที่มีค่า NaN หลังจากแปลง
train_data = train_data.dropna(subset=['PM2.5'])
test_data = test_data.dropna(subset=['PM2.5'])

# เตรียม Training และ Test Set
X_train, y_train = create_sequences(train_data['PM2.5'].values, n_steps)
X_test, y_test = create_sequences(test_data['PM2.5'].values, n_steps)

# ตรวจสอบการจัดการกับ NaN ในชุดข้อมูล
X_train = np.nan_to_num(X_train)  # แปลง NaN เป็น 0
y_train = np.nan_to_num(y_train)
X_test = np.nan_to_num(X_test)
y_test = np.nan_to_num(y_test)

# Reshape สำหรับ RNN ให้เป็น 3D (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # เพิ่มมิติที่ 3 สำหรับ features
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))  # เพิ่มมิติที่ 3 สำหรับ features

# สร้างโมเดล RNN
model_rnn = Sequential([
    SimpleRNN(64, activation='relu', input_shape=(n_steps, 1)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer สำหรับค่าตัวเลขเดียว
])

# คอมไพล์โมเดล
model_rnn.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

# เทรนโมเดล
model_rnn.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# ทำนายค่าจาก Test Set
y_pred = model_rnn.predict(X_test)

# คำนวณ MSE, RMSE, MAE, MAPE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) *100

# แสดงผลลัพธ์
print(f'MSE: {mse}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}')

# เพิ่มข้อมูลที่ทำนายล่าสุดในแต่ละวันจาก Test Set เข้า Training Set
test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])  # แปลงเป็น datetime
latest_per_day = test_data.loc[test_data.groupby(test_data['วันที่เต็ม'].dt.date)['วันที่เต็ม'].idxmax()]  # เลือกข้อมูลล่าสุดของแต่ละวัน

# เลือกคอลัมน์ทั้งหมดสำหรับการอัปเดต Training Set
columns_to_include = ['สถานี', 'วันที่เต็ม', 'ปริมาณฝน', 'ความชื้นสัมพัทธ์', 'ความเร็วลม', 'ทิศทางลม', 'หมอก', 'อุณหภูมิ', 'PM10', 'PM2.5']
latest_per_day = latest_per_day[columns_to_include]  # เก็บเฉพาะคอลัมน์ที่ต้องการ

# รวมข้อมูลเข้ากับ Training Set
updated_train_data = pd.concat([train_data[columns_to_include], latest_per_day], ignore_index=True)

# จัดการค่า NaN ใน Training Set ใหม่
updated_train_data = updated_train_data.dropna(subset=['PM2.5'])  # ลบแถวที่ไม่มี PM2.5
updated_train_data.fillna(0, inplace=True)  # แทนที่ค่า NaN อื่น ๆ ด้วย 0

# เตรียมข้อมูลใหม่สำหรับการเทรน
X_train_new, y_train_new = create_sequences(updated_train_data['PM2.5'].values, n_steps)

# แปลงข้อมูลใหม่สำหรับการเทรน
X_train_new = np.nan_to_num(X_train_new)
y_train_new = np.nan_to_num(y_train_new)

# Reshape สำหรับ RNN
X_train_new = X_train_new.reshape((X_train_new.shape[0], X_train_new.shape[1], 1))

# เทรนโมเดลใหม่ด้วย Training Set ที่อัปเดตแล้ว
model_rnn.fit(X_train_new, y_train_new, epochs=10, batch_size=32)

# เพิ่มฟิลด์ Actual และ Predicted สำหรับการประเมิน
results = pd.DataFrame({
    'Actual': y_test.flatten(),
    'Predicted': y_pred.flatten()
})

# เลือกคอลัมน์ที่ต้องการจาก test_data
columns_to_include = ['สถานี', 'วันที่เต็ม', 'ปริมาณฝน', 'ความชื้นสัมพัทธ์', 'ความเร็วลม', 'ทิศทางลม', 'หมอก', 'อุณหภูมิ', 'PM10']

# หาค่าล่าสุดของแต่ละวันในชุดข้อมูล test_data
test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])
latest_per_day = test_data.loc[test_data.groupby(test_data['วันที่เต็ม'].dt.date)['วันที่เต็ม'].idxmax()]

# เลือกเฉพาะคอลัมน์ที่ต้องการ
latest_per_day = latest_per_day[columns_to_include]

# รวมข้อมูลล่าสุดของแต่ละวันเข้ากับผลลัพธ์การทำนาย
final_results = pd.concat([results, latest_per_day.reset_index(drop=True)], axis=1)

# บันทึกผลลัพธ์ที่รวมข้อมูลลงในไฟล์ CSV
final_results.to_csv('/content/drive/My Drive/Data/Predictions_with_Fields_RNN.csv', index=False)

# บันทึก Training Set ที่อัปเดตแล้ว
updated_train_data.to_csv('/content/drive/My Drive/Data/Updated_Training_Set_RNN.csv', index=False)

# บันทึกโมเดลที่ฝึกแล้วเป็นไฟล์ H5
model_rnn.save('/content/drive/My Drive/Data/RNN_Model.h5')

print("ผลลัพธ์การทำนายพร้อมฟิลด์ที่เพิ่มเข้าไปถูกบันทึกแล้ว!")
print("โมเดลได้ถูกบันทึกในไฟล์ H5 แล้ว!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


69/69 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 105.5306 - mae: 7.2349 - mape: 32.9252 - val_loss: 51.7920 - val_mae: 4.9330 - val_mape: 19.7778
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 48.4244 - mae: 4.7147 - mape: 21.8111 - val_loss: 49.9837 - val_mae: 4.8578 - val_mape: 19.3332
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39.8056 - mae: 4.4565 - mape: 21.6374 - val_loss: 45.9937 - val_mae: 4.6303 - val_mape: 18.8935
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 45.2146 - mae: 4.6099 - mape: 21.8482 - val_loss: 45.6641 - val_mae: 4.6915 - val_mape: 19.9645
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 38.8946 - mae: 4.3881 - mape: 21.5635 - val_loss: 44.6236 - val_mae: 4.5799 - val_mape: 18.9978
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 47.6099 - mae: 4.5471 - mape: 21.7030 - val_loss: 47.0185 - val_mae: 4.7988 - val_mape: 20.7773
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 42.7723 - mae: 4.5439 - map

<ipython-input-20-fa1ade5f81d6>:81: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])  # แปลงเป็น datetime


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 40.9670 - mae: 4.5485 - mape: 22.2728
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 41.2729 - mae: 4.5485 - mape: 22.6088
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39.0163 - mae: 4.3928 - mape: 21.6204
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 40.4184 - mae: 4.4440 - mape: 21.5466
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 40.1066 - mae: 4.4457 - mape: 22.3823
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 41.1872 - mae: 4.5203 - mape: 22.1572
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 42.4531 - mae: 4.4656 - mape: 21.8247
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 40.5996 - mae: 4.4406 - mape: 21.8078
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 41.0154 - mae: 4.5214 - mape: 22.2449
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 46.5868 - mae: 4.5327 - mape: 21.7207


ผลลัพธ์การทำนายพร้อมฟิลด์ที่เพิ่มเข้าไปถูกบันทึกแล้ว!
โมเดลได้ถูกบันทึกในไฟล์ H5 แล้ว!


LSTM

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from google.colab import drive

# เชื่อมต่อกับ Google Drive
drive.mount('/content/drive')

# ตั้งที่อยู่ของไฟล์
train_path = '/content/drive/My Drive/Data/Trainning_Set.csv'
test_path = '/content/drive/My Drive/Data/Test_Set.csv'

# อ่านไฟล์ CSV
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# แปลง PM2.5 เป็น Sliding Window
def create_sequences(data, n_steps):
    sequences = []
    labels = []
    for i in range(len(data) - n_steps):
        seq = data[i:i + n_steps]
        label = data[i + n_steps]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# กำหนดจำนวนวันที่ต้องการดูย้อนหลัง
n_steps = 7

# ตรวจสอบให้แน่ใจว่า 'PM2.5' เป็นตัวเลข
train_data['PM2.5'] = pd.to_numeric(train_data['PM2.5'], errors='coerce')
test_data['PM2.5'] = pd.to_numeric(test_data['PM2.5'], errors='coerce')

# ลบแถวที่มีค่า NaN หลังจากแปลง
train_data = train_data.dropna(subset=['PM2.5'])
test_data = test_data.dropna(subset=['PM2.5'])

# เตรียม Training และ Test Set
X_train, y_train = create_sequences(train_data['PM2.5'].values, n_steps)
X_test, y_test = create_sequences(test_data['PM2.5'].values, n_steps)

# ตรวจสอบการจัดการกับ NaN ในชุดข้อมูล
X_train = np.nan_to_num(X_train)  # แปลง NaN เป็น 0
y_train = np.nan_to_num(y_train)
X_test = np.nan_to_num(X_test)
y_test = np.nan_to_num(y_test)

# Reshape สำหรับ LSTM ให้เป็น 3D (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # เพิ่มมิติที่ 3 สำหรับ features
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))  # เพิ่มมิติที่ 3 สำหรับ features

# สร้างโมเดล LSTM
model_lstm = Sequential([
    LSTM(64, activation='relu', input_shape=(n_steps, 1)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer สำหรับค่าตัวเลขเดียว
])

# คอมไพล์โมเดล
model_lstm.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

# เทรนโมเดล
model_lstm.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# ทำนายค่าจาก Test Set
y_pred = model_lstm.predict(X_test)

# คำนวณ MSE, RMSE, MAE, MAPE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)*100

# แสดงผลลัพธ์
print(f'MSE: {mse}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}')

# เพิ่มข้อมูลที่ทำนายล่าสุดในแต่ละวันจาก Test Set เข้า Training Set
test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])  # แปลงเป็น datetime
latest_per_day = test_data.loc[test_data.groupby(test_data['วันที่เต็ม'].dt.date)['วันที่เต็ม'].idxmax()]  # เลือกข้อมูลล่าสุดของแต่ละวัน

# เลือกคอลัมน์ทั้งหมดสำหรับการอัปเดต Training Set
columns_to_include = ['สถานี', 'วันที่เต็ม', 'ปริมาณฝน', 'ความชื้นสัมพัทธ์', 'ความเร็วลม', 'ทิศทางลม', 'หมอก', 'อุณหภูมิ', 'PM10', 'PM2.5']
latest_per_day = latest_per_day[columns_to_include]  # เก็บเฉพาะคอลัมน์ที่ต้องการ

# รวมข้อมูลเข้ากับ Training Set
updated_train_data = pd.concat([train_data[columns_to_include], latest_per_day], ignore_index=True)

# จัดการค่า NaN ใน Training Set ใหม่
updated_train_data = updated_train_data.dropna(subset=['PM2.5'])  # ลบแถวที่ไม่มี PM2.5
updated_train_data.fillna(0, inplace=True)  # แทนที่ค่า NaN อื่น ๆ ด้วย 0

# เตรียมข้อมูลใหม่สำหรับการเทรน
X_train_new, y_train_new = create_sequences(updated_train_data['PM2.5'].values, n_steps)

# แปลงข้อมูลใหม่สำหรับการเทรน
X_train_new = np.nan_to_num(X_train_new)
y_train_new = np.nan_to_num(y_train_new)

# Reshape สำหรับ LSTM
X_train_new = X_train_new.reshape((X_train_new.shape[0], X_train_new.shape[1], 1))

# เทรนโมเดลใหม่ด้วย Training Set ที่อัปเดตแล้ว
model_lstm.fit(X_train_new, y_train_new, epochs=10, batch_size=32)

# เพิ่มฟิลด์ Actual และ Predicted สำหรับการประเมิน
results = pd.DataFrame({
    'Actual': y_test.flatten(),
    'Predicted': y_pred.flatten()
})

# เลือกคอลัมน์ที่ต้องการจาก test_data
columns_to_include = ['สถานี', 'วันที่เต็ม', 'ปริมาณฝน', 'ความชื้นสัมพัทธ์', 'ความเร็วลม', 'ทิศทางลม', 'หมอก', 'อุณหภูมิ', 'PM10']

# หาค่าล่าสุดของแต่ละวันในชุดข้อมูล test_data
test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])
latest_per_day = test_data.loc[test_data.groupby(test_data['วันที่เต็ม'].dt.date)['วันที่เต็ม'].idxmax()]

# เลือกเฉพาะคอลัมน์ที่ต้องการ
latest_per_day = latest_per_day[columns_to_include]

# รวมข้อมูลล่าสุดของแต่ละวันเข้ากับผลลัพธ์การทำนาย
final_results = pd.concat([results, latest_per_day.reset_index(drop=True)], axis=1)

# บันทึกผลลัพธ์ที่รวมข้อมูลลงในไฟล์ CSV
final_results.to_csv('/content/drive/My Drive/Data/Predictions_with_Fields_LSTM.csv', index=False)

# บันทึก Training Set ที่อัปเดตแล้ว
updated_train_data.to_csv('/content/drive/My Drive/Data/Updated_Training_Set_LSTM.csv', index=False)

# บันทึกโมเดลที่ฝึกแล้วเป็นไฟล์ H5
model_lstm.save('/content/drive/My Drive/Data/LSTM_Model.h5')

print("ผลลัพธ์การทำนายพร้อมฟิลด์ที่เพิ่มเข้าไปถูกบันทึกแล้ว!")
print("โมเดลได้ถูกบันทึกในไฟล์ H5 แล้ว!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


69/69 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - loss: 379.7740 - mae: 15.1641 - mape: 70.8099 - val_loss: 58.1540 - val_mae: 4.8713 - val_mape: 19.1525
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 54.8281 - mae: 5.0085 - mape: 24.3689 - val_loss: 50.9215 - val_mae: 4.6742 - val_mape: 18.8975
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 47.5639 - mae: 4.6196 - mape: 22.9217 - val_loss: 46.0117 - val_mae: 4.6373 - val_mape: 19.5108
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 50.0401 - mae: 4.7786 - mape: 22.9770 - val_loss: 50.2241 - val_mae: 4.6816 - val_mape: 18.2504
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 42.9671 - mae: 4.5870 - mape: 23.0159 - val_loss: 47.8579 - val_mae: 4.9571 - val_mape: 22.7425
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 43.2314 - mae: 4.3730 - mape: 22.3080 - val_loss: 46.5366 - val_mae: 4.6377 - val_mape: 19.5678
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 45.1816 - mae: 4.6499 - ma

<ipython-input-21-b501d17bf71b>:81: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])  # แปลงเป็น datetime


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 43.2580 - mae: 4.6069 - mape: 22.6842
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 48.1649 - mae: 4.5781 - mape: 22.2305
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 38.6608 - mae: 4.3752 - mape: 21.5858
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 42.2743 - mae: 4.5244 - mape: 21.3577
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 37.5638 - mae: 4.3404 - mape: 22.1034
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 38.1770 - mae: 4.4048 - mape: 21.8990
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 37.4964 - mae: 4.2896 - mape: 21.1826
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39.4261 - mae: 4.3995 - mape: 21.8350
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 42.4813 - mae: 4.4906 - mape: 21.4260
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 38.6134 - mae: 4.2456 - mape: 20.8922


ผลลัพธ์การทำนายพร้อมฟิลด์ที่เพิ่มเข้าไปถูกบันทึกแล้ว!
โมเดลได้ถูกบันทึกในไฟล์ H5 แล้ว!


GRU

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from google.colab import drive

# เชื่อมต่อกับ Google Drive
drive.mount('/content/drive')

# ตั้งที่อยู่ของไฟล์
train_path = '/content/drive/My Drive/Data/Trainning_Set.csv'
test_path = '/content/drive/My Drive/Data/Test_Set.csv'

# อ่านไฟล์ CSV
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# แปลง PM2.5 เป็น Sliding Window
def create_sequences(data, n_steps):
    sequences = []
    labels = []
    for i in range(len(data) - n_steps):
        seq = data[i:i + n_steps]
        label = data[i + n_steps]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# กำหนดจำนวนวันที่ต้องการดูย้อนหลัง
n_steps = 7

# ตรวจสอบให้แน่ใจว่า 'PM2.5' เป็นตัวเลข
train_data['PM2.5'] = pd.to_numeric(train_data['PM2.5'], errors='coerce')
test_data['PM2.5'] = pd.to_numeric(test_data['PM2.5'], errors='coerce')

# ลบแถวที่มีค่า NaN หลังจากแปลง
train_data = train_data.dropna(subset=['PM2.5'])
test_data = test_data.dropna(subset=['PM2.5'])

# เตรียม Training และ Test Set
X_train, y_train = create_sequences(train_data['PM2.5'].values, n_steps)
X_test, y_test = create_sequences(test_data['PM2.5'].values, n_steps)

# ตรวจสอบการจัดการกับ NaN ในชุดข้อมูล
X_train = np.nan_to_num(X_train)  # แปลง NaN เป็น 0
y_train = np.nan_to_num(y_train)
X_test = np.nan_to_num(X_test)
y_test = np.nan_to_num(y_test)

# Reshape สำหรับ GRU ให้เป็น 3D (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # เพิ่มมิติที่ 3 สำหรับ features
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))  # เพิ่มมิติที่ 3 สำหรับ features

# สร้างโมเดล GRU
model_gru = Sequential([
    GRU(64, activation='relu', input_shape=(n_steps, 1)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer สำหรับค่าตัวเลขเดียว
])

# คอมไพล์โมเดล
model_gru.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

# เทรนโมเดล
model_gru.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# ทำนายค่าจาก Test Set
y_pred = model_gru.predict(X_test)

# คำนวณ MSE, RMSE, MAE, MAPE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)*100

# แสดงผลลัพธ์
print(f'MSE: {mse}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}')

# เพิ่มข้อมูลที่ทำนายล่าสุดในแต่ละวันจาก Test Set เข้า Training Set
test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])  # แปลงเป็น datetime
latest_per_day = test_data.loc[test_data.groupby(test_data['วันที่เต็ม'].dt.date)['วันที่เต็ม'].idxmax()]  # เลือกข้อมูลล่าสุดของแต่ละวัน

# เลือกคอลัมน์ทั้งหมดสำหรับการอัปเดต Training Set
columns_to_include = ['สถานี', 'วันที่เต็ม', 'ปริมาณฝน', 'ความชื้นสัมพัทธ์', 'ความเร็วลม', 'ทิศทางลม', 'หมอก', 'อุณหภูมิ', 'PM10', 'PM2.5']
latest_per_day = latest_per_day[columns_to_include]  # เก็บเฉพาะคอลัมน์ที่ต้องการ

# รวมข้อมูลเข้ากับ Training Set
updated_train_data = pd.concat([train_data[columns_to_include], latest_per_day], ignore_index=True)

# จัดการค่า NaN ใน Training Set ใหม่
updated_train_data = updated_train_data.dropna(subset=['PM2.5'])  # ลบแถวที่ไม่มี PM2.5
updated_train_data.fillna(0, inplace=True)  # แทนที่ค่า NaN อื่น ๆ ด้วย 0

# เตรียมข้อมูลใหม่สำหรับการเทรน
X_train_new, y_train_new = create_sequences(updated_train_data['PM2.5'].values, n_steps)

# แปลงข้อมูลใหม่สำหรับการเทรน
X_train_new = np.nan_to_num(X_train_new)
y_train_new = np.nan_to_num(y_train_new)

# Reshape สำหรับ GRU
X_train_new = X_train_new.reshape((X_train_new.shape[0], X_train_new.shape[1], 1))

# เทรนโมเดลใหม่ด้วย Training Set ที่อัปเดตแล้ว
model_gru.fit(X_train_new, y_train_new, epochs=10, batch_size=32)

# เพิ่มฟิลด์ Actual และ Predicted สำหรับการประเมิน
results = pd.DataFrame({
    'Actual': y_test.flatten(),
    'Predicted': y_pred.flatten()
})

# เลือกคอลัมน์ที่ต้องการจาก test_data
columns_to_include = ['สถานี', 'วันที่เต็ม', 'ปริมาณฝน', 'ความชื้นสัมพัทธ์', 'ความเร็วลม', 'ทิศทางลม', 'หมอก', 'อุณหภูมิ', 'PM10']

# หาค่าล่าสุดของแต่ละวันในชุดข้อมูล test_data
test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])
latest_per_day = test_data.loc[test_data.groupby(test_data['วันที่เต็ม'].dt.date)['วันที่เต็ม'].idxmax()]

# เลือกเฉพาะคอลัมน์ที่ต้องการ
latest_per_day = latest_per_day[columns_to_include]

# รวมข้อมูลล่าสุดของแต่ละวันเข้ากับผลลัพธ์การทำนาย
final_results = pd.concat([results, latest_per_day.reset_index(drop=True)], axis=1)

# บันทึกผลลัพธ์ที่รวมข้อมูลลงในไฟล์ CSV
final_results.to_csv('/content/drive/My Drive/Data/Predictions_with_Fields_GRU.csv', index=False)

# บันทึก Training Set ที่อัปเดตแล้ว
updated_train_data.to_csv('/content/drive/My Drive/Data/Updated_Training_Set_GRU.csv', index=False)

print("ผลลัพธ์การทำนายพร้อมฟิลด์ที่เพิ่มเข้าไปถูกบันทึกแล้ว!")

# บันทึกโมเดล GRU ที่ฝึกเสร็จแล้วเป็นไฟล์ .h5
model_gru.save('/content/drive/My Drive/Data/GRU_PM25_Model.h5')
print("โมเดล GRU ได้ถูกบันทึกเป็นไฟล์ .h5 เรียบร้อยแล้ว!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


69/69 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 475.4847 - mae: 17.7584 - mape: 79.4257 - val_loss: 57.2416 - val_mae: 5.0465 - val_mape: 21.0174
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 56.5386 - mae: 5.0690 - mape: 25.2868 - val_loss: 53.8017 - val_mae: 4.9052 - val_mape: 20.2430
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 49.1976 - mae: 4.7644 - mape: 24.3316 - val_loss: 48.2645 - val_mae: 4.9806 - val_mape: 22.5797
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 44.8901 - mae: 4.6743 - mape: 23.9737 - val_loss: 45.7960 - val_mae: 4.8452 - val_mape: 21.6730
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 44.2691 - mae: 4.6458 - mape: 23.2206 - val_loss: 44.1575 - val_mae: 4.7044 - val_mape: 20.6127
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 42.7890 - mae: 4.5565 - mape: 22.2960 - val_loss: 43.1271 - val_mae: 4.5946 - val_mape: 19.9035
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 42.5694 - mae: 4.5774 - ma

<ipython-input-27-02cff3993dd0>:81: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])  # แปลงเป็น datetime


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 39.6507 - mae: 4.4197 - mape: 21.9050
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 41.9008 - mae: 4.4616 - mape: 21.8216
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 39.4242 - mae: 4.4813 - mape: 22.2465
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 38.6296 - mae: 4.3742 - mape: 21.2986
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 40.0822 - mae: 4.3606 - mape: 21.2885
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 37.2152 - mae: 4.2365 - mape: 20.9980
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 37.3253 - mae: 4.3284 - mape: 21.2981
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 44.4075 - mae: 4.4598 - mape: 21.9472
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 49.8161 - mae: 4.6398 - mape: 22.7070
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 44.9335 - mae: 4.5292 - mape: 21.7681


ผลลัพธ์การทำนายพร้อมฟิลด์ที่เพิ่มเข้าไปถูกบันทึกแล้ว!
โมเดล GRU ได้ถูกบันทึกเป็นไฟล์ .h5 เรียบร้อยแล้ว!


MLP

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

from google.colab import drive
drive.mount('/content/drive')

# ตั้งที่อยู่ของไฟล์
train_path = '/content/drive/My Drive/Data/Trainning_Set.csv'
test_path = '/content/drive/My Drive/Data/Test_Set.csv'

# อ่านไฟล์ CSV
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# แปลง PM2.5 เป็น Sliding Window
def create_sequences(data, n_steps):
    sequences = []
    labels = []
    for i in range(len(data) - n_steps):
        seq = data[i:i + n_steps]
        label = data[i + n_steps]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# กำหนดจำนวนวันที่ต้องการดูย้อนหลัง
n_steps = 7

# ตรวจสอบให้แน่ใจว่า 'PM2.5' เป็นตัวเลข
train_data['PM2.5'] = pd.to_numeric(train_data['PM2.5'], errors='coerce')
test_data['PM2.5'] = pd.to_numeric(test_data['PM2.5'], errors='coerce')

# ลบแถวที่มีค่า NaN หลังจากแปลง
train_data = train_data.dropna(subset=['PM2.5'])
test_data = test_data.dropna(subset=['PM2.5'])

# เตรียม Training และ Test Set
X_train, y_train = create_sequences(train_data['PM2.5'].values, n_steps)
X_test, y_test = create_sequences(test_data['PM2.5'].values, n_steps)

# ตรวจสอบการจัดการกับ NaN ในชุดข้อมูล
X_train = np.nan_to_num(X_train)  # แปลง NaN เป็น 0
y_train = np.nan_to_num(y_train)
X_test = np.nan_to_num(X_test)
y_test = np.nan_to_num(y_test)

# Reshape สำหรับ MLP ให้เป็น 2D (samples, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]))  # ไม่มีมิติที่ 3
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1]))

# สร้างโมเดล MLP
model_mlp = Sequential([
    Dense(128, activation='relu', input_shape=(n_steps,)),  # Hidden Layer 1
    Dense(64, activation='relu'),  # Hidden Layer 2
    Dense(1)  # Output layer สำหรับค่าตัวเลขเดียว
])

# คอมไพล์โมเดล
model_mlp.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

# เทรนโมเดล
model_mlp.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# ทำนายค่าจาก Test Set
y_pred = model_mlp.predict(X_test)

# คำนวณ MSE, RMSE, MAE, MAPE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)*100

# แสดงผลลัพธ์
print(f'MSE: {mse}, RMSE: {rmse}, MAE: {mae}, MAPE: {mape}')

# เพิ่มข้อมูลที่ทำนายล่าสุดในแต่ละวันจาก Test Set เข้า Training Set
test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])  # แปลงเป็น datetime
latest_per_day = test_data.loc[test_data.groupby(test_data['วันที่เต็ม'].dt.date)['วันที่เต็ม'].idxmax()]  # เลือกข้อมูลล่าสุดของแต่ละวัน

# เลือกคอลัมน์ทั้งหมดสำหรับการอัปเดต Training Set
columns_to_include = ['สถานี', 'วันที่เต็ม', 'ปริมาณฝน', 'ความชื้นสัมพัทธ์', 'ความเร็วลม', 'ทิศทางลม', 'หมอก', 'อุณหภูมิ', 'PM10', 'PM2.5']
latest_per_day = latest_per_day[columns_to_include]  # เก็บเฉพาะคอลัมน์ที่ต้องการ

# รวมข้อมูลเข้ากับ Training Set
updated_train_data = pd.concat([train_data[columns_to_include], latest_per_day], ignore_index=True)

# จัดการค่า NaN ใน Training Set ใหม่
updated_train_data = updated_train_data.dropna(subset=['PM2.5'])  # ลบแถวที่ไม่มี PM2.5
updated_train_data.fillna(0, inplace=True)  # แทนที่ค่า NaN อื่น ๆ ด้วย 0

# เตรียมข้อมูลใหม่สำหรับการเทรน
X_train_new, y_train_new = create_sequences(updated_train_data['PM2.5'].values, n_steps)

# แปลงข้อมูลใหม่สำหรับการเทรน
X_train_new = np.nan_to_num(X_train_new)
y_train_new = np.nan_to_num(y_train_new)

# Reshape สำหรับ MLP
X_train_new = X_train_new.reshape((X_train_new.shape[0], X_train_new.shape[1]))

# เทรนโมเดลใหม่ด้วย Training Set ที่อัปเดตแล้ว
model_mlp.fit(X_train_new, y_train_new, epochs=10, batch_size=32)

# เพิ่มฟิลด์ Actual และ Predicted สำหรับการประเมิน
results = pd.DataFrame({
    'Actual': y_test.flatten(),
    'Predicted': y_pred.flatten()
})

# เลือกคอลัมน์ที่ต้องการจาก test_data
columns_to_include = ['สถานี', 'วันที่เต็ม', 'ปริมาณฝน', 'ความชื้นสัมพัทธ์', 'ความเร็วลม', 'ทิศทางลม', 'หมอก', 'อุณหภูมิ', 'PM10']

# หาค่าล่าสุดของแต่ละวันในชุดข้อมูล test_data
test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])
latest_per_day = test_data.loc[test_data.groupby(test_data['วันที่เต็ม'].dt.date)['วันที่เต็ม'].idxmax()]

# เลือกเฉพาะคอลัมน์ที่ต้องการ
latest_per_day = latest_per_day[columns_to_include]

# รวมข้อมูลล่าสุดของแต่ละวันเข้ากับผลลัพธ์การทำนาย
final_results = pd.concat([results, latest_per_day.reset_index(drop=True)], axis=1)

# บันทึกผลลัพธ์ที่รวมข้อมูลลงในไฟล์ CSV
final_results.to_csv('/content/drive/My Drive/Data/Predictions_with_Fields_MLP.csv', index=False)

# บันทึก Training Set ที่อัปเดตแล้ว
updated_train_data.to_csv('/content/drive/My Drive/Data/Updated_Training_Set_MLP.csv', index=False)

# บันทึกโมเดลที่ฝึกแล้วเป็นไฟล์ H5
model_mlp.save('/content/drive/My Drive/Data/MLP_Model.h5')

print("ผลลัพธ์การทำนายพร้อมฟิลด์ที่เพิ่มเข้าไปถูกบันทึกแล้ว!")
print("โมเดลได้ถูกบันทึกในไฟล์ H5 แล้ว!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 135.7889 - mae: 8.0771 - mape: 38.0053 - val_loss: 49.8387 - val_mae: 4.9299 - val_mape: 21.2944
Epoch 2/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 55.0902 - mae: 4.9915 - mape: 23.7729 - val_loss: 46.4940 - val_mae: 4.7198 - val_mape: 20.0021
Epoch 3/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 42.7813 - mae: 4.5073 - mape: 21.8905 - val_loss: 45.5367 - val_mae: 4.6356 - val_mape: 19.0564
Epoch 4/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 44.8409 - mae: 4.5654 - mape: 21.9217 - val_loss: 53.9596 - val_mae: 5.0036 - val_mape: 19.6001
Epoch 5/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 42.8862 - mae: 4.6421 - mape: 22.1454 - val_loss: 50.1312 - val_mae: 4.9507 - val_mape: 22.1503
Epoch 6/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 47.3602 - mae: 4.6539 - mape: 22.9748 - val_loss: 45.2166 - val_mae: 4.6654 - val_mape: 19.7312
Epoch 7/50
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43.3381 - mae: 4.5851 - map

<ipython-input-25-4c82826bd522>:80: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  test_data['วันที่เต็ม'] = pd.to_datetime(test_data['วันที่เต็ม'])  # แปลงเป็น datetime


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 46.3353 - mae: 4.6494 - mape: 22.3429
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 36.9400 - mae: 4.2554 - mape: 21.3717
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 43.9181 - mae: 4.5351 - mape: 21.9797
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38.2627 - mae: 4.4259 - mape: 22.0881
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 42.9534 - mae: 4.5448 - mape: 22.2576
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 44.5215 - mae: 4.6448 - mape: 22.9607
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39.8086 - mae: 4.3872 - mape: 21.1277
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 40.5207 - mae: 4.5378 - mape: 22.5386
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 41.8244 - mae: 4.5738 - mape: 22.0264
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 38.6514 - mae: 4.3257 - mape: 21.2613


ผลลัพธ์การทำนายพร้อมฟิลด์ที่เพิ่มเข้าไปถูกบันทึกแล้ว!
โมเดลได้ถูกบันทึกในไฟล์ H5 แล้ว!
